# [STARTER] Exercise - Create an Agent with external API call enabled

In this exercise, you'll build an agent that can interact with external APIs to gather real-time data
and provide responses based on that information. You'll combine concepts from state management and
memory while adding the ability to make external API calls safely and effectively.


## Challenge

Your task is to create an agent that can make External API Calls:

- Implement tools that interact with real APIs
- Handle API responses and errors gracefully
- Use environment variables for API keys
- Process and format API data for user consumption

## Setup
First, let's import the necessary libraries:

In [1]:
import os
import random
from typing import List
import requests
from dotenv import load_dotenv

from lib.agents import Agent
from lib.messages import BaseMessage
from lib.tooling import tool

In [2]:
load_dotenv()

True

## Define API tools

Feel free to use any open service available through APIs.

Here are a few examples. You can follow the instructions given.
- https://jsonplaceholder.typicode.com/guide/
- https://www.exchangerate-api.com/
- https://openweathermap.org/

Or you can find one you're interested in here.
- https://github.com/public-apis/public-apis

In [3]:
# TODO: Define as many tools that access external APIs as you want
# Example:
@tool
def get_random_pokemon() -> dict:
    """Get a random Pokemon from the original 151"""
    URL = "https://pokeapi.co/api/v2/pokemon?limit=151"
    response = requests.get(URL)
    response.raise_for_status()
    return random.choice(response.json()['results'])

In [4]:
# TODO: Add all the tools you have defined
tools = [get_random_pokemon]

In [5]:
# TODO: Add instructions to your agent

agent = Agent(
    model_name="gpt-4o-mini",
    instructions=(
        "You are an assistant that can help with:\n"
        "1. Getting a random Pokemon:\n"
        "Use the available tools to help answer the question about the topics\n"
        "Maintain the context across conversation within the same session."
    ),
    tools=tools
)

In [6]:
def print_messages(messages: List[BaseMessage]):
    for m in messages:
        print(f" -> (role = {m.role}, content = {m.content}, tool_calls = {getattr(m, 'tool_calls', None)})")

## Run your Agent

In [7]:
# TODO: Change the query and then run your agent
query="Pick one random pokemon?", 
session_id = "external_tools"

In [9]:
run1 = agent.invoke(
    query="Pick two random pokemon?", 
    session_id=session_id,
)

print("\nMessages from run 1:")
messages = run1.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 1:
 -> (role = system, content = You are an assistant that can help with:
1. Getting a random Pokemon:
Use the available tools to help answer the question about the topics
Maintain the context across conversation within the same session., tool_calls = None)
 -> (role = user, content = Pick one random pokemon?, tool_calls = None)
 -> (role = assistant, content = None, tool_calls = [ChatCompletionMessageToolCall(id='call_AxkjI0S2NdpSNUAJU9SaLMBw', function=Function(arguments='{}', name='get_random_pokemon'), type='function')])
 -> (role = tool, content = "{'name': 'charmeleon', 'url': 'https://pokeapi.co/api/v2/pokemon/5/'}", tool_calls = None)
 -> (role = assistant, content = I randomly picked a Pokémon for you: **

## Check session histories

In [10]:
runs = agent.get_session_runs(session_id)
for i, run_object in enumerate(runs, 1):
    print(f"\n# Run {i}", run_object.metadata)
    print("Messages:")
    print_messages(run_object.get_final_state()["messages"])


# Run 1 {'run_id': '872d8b58-b7ce-42ee-b15e-ed8ce74b866d', 'start_timestamp': '2025-12-03 00:47:25.306799', 'end_timestamp': '2025-12-03 00:47:28.464048', 'snapshot_counts': 5}
Messages:
 -> (role = system, content = You are an assistant that can help with:
1. Getting a random Pokemon:
Use the available tools to help answer the question about the topics
Maintain the context across conversation within the same session., tool_calls = None)
 -> (role = user, content = Pick one random pokemon?, tool_calls = None)
 -> (role = assistant, content = None, tool_calls = [ChatCompletionMessageToolCall(id='call_AxkjI0S2NdpSNUAJU9SaLMBw', function=Function(arguments='{}', name='get_random_pokemon'), type='function')])
 -> (role = tool, content = "{'name': 'charmeleon', 'url': 'https://pokeapi.co/api/v2/pokemon/5/'}", tool_calls = None)
 -> (role = assistant, content = I randomly picked a Pokémon for you: **Charmeleon**! 

You can find more about it [here](https://pokeapi.co/api/v2/pokemon/5/)., to

In [11]:
runs = agent.get_session_runs(session_id)
for run_object in runs:
    print(run_object)
    for snp in run_object.snapshots:
        print(f"-> {snp}")
    print("\n")

Run('872d8b58-b7ce-42ee-b15e-ed8ce74b866d')
-> Snapshot('8c8163bd-6495-4c8d-b767-5a6d0a2cdb8f') @ [2025-12-03 00:47:25.306945]: __entry__.State({'user_query': 'Pick one random pokemon?', 'instructions': 'You are an assistant that can help with:\n1. Getting a random Pokemon:\nUse the available tools to help answer the question about the topics\nMaintain the context across conversation within the same session.', 'messages': [], 'current_tool_calls': None, 'session_id': 'external_tools'})
-> Snapshot('ce8c42d8-707b-4a03-9a6c-01e17a3d61ea') @ [2025-12-03 00:47:25.307097]: message_prep.State({'user_query': 'Pick one random pokemon?', 'instructions': 'You are an assistant that can help with:\n1. Getting a random Pokemon:\nUse the available tools to help answer the question about the topics\nMaintain the context across conversation within the same session.', 'messages': [SystemMessage(role='system', content='You are an assistant that can help with:\n1. Getting a random Pokemon:\nUse the avail